# Лабораторная работа №7

## Рекуррентные нейронные сети для анализа текста

Набор данных для предсказания оценок для отзывов, собранных с сайта _imdb.com_, который состоит из 50,000 отзывов в виде текстовых файлов.

Отзывы разделены на положительные (25,000) и отрицательные (25,000).

Данные предварительно токенизированы по принципу «мешка слов», индексы слов можно взять из словаря (_imdb.vocab_).

Обучающая выборка включает в себя 12,500 положительных и 12,500 отрицательных отзывов, контрольная выборка также содержит 12,500 положительных и 12,500 отрицательных отзывов.

Данные можно скачать ~~на сайте _Kaggle_~~: ~~https://www.kaggle.com/iarunava/imdb-movie-reviews-dataset~~ https://ai.stanford.edu/~amaas/data/sentiment/

### Задание 1

Загрузите данные. Преобразуйте текстовые файлы во внутренние структуры данных, которые используют индексы вместо слов.

In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [0]:
BASE_DIR = '/content/drive/My Drive/Colab Files/mo-2'

import sys

sys.path.append(BASE_DIR)

import os

In [0]:
DATA_ARCHIVE_NAME = 'imdb-dataset-of-50k-movie-reviews.zip'

LOCAL_DIR_NAME = 'imdb-sentiments'

In [0]:
from zipfile import ZipFile

with ZipFile(os.path.join(BASE_DIR, DATA_ARCHIVE_NAME), 'r') as zip_:
    zip_.extractall(LOCAL_DIR_NAME)

In [0]:
DATA_FILE_PATH = 'imdb-sentiments/IMDB Dataset.csv'

In [0]:
import pandas as pd

all_df = pd.read_csv(DATA_FILE_PATH)

In [7]:
print(all_df.shape)

(50000, 2)


In [0]:
train_df = all_df.sample(frac = 0.7)
all_df = all_df.drop(train_df.index)

val_df = all_df.sample(frac = 0.5)
test_df = all_df.drop(val_df.index)

In [9]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(35000, 2)
(7500, 2)
(7500, 2)


In [10]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
MAX_LENGTH = 200

STRING_DTYPE = '<U12'

PADDING_TOKEN = 'PAD'

LIMIT_OF_TOKENS = 100000

In [0]:
from nltk import word_tokenize
import numpy as np
import string
import re

def tokenize_string(_string):
    return  [tok_.lower() for tok_ in word_tokenize(_string) if not re.fullmatch('[' + string.punctuation + ']+', tok_)]

def pad(A, length):
    arr = np.empty(length, dtype = STRING_DTYPE)
    arr.fill(PADDING_TOKEN)
    arr[:len(A)] = A
    return arr

def tokenize_row(_sentence):
    return pad(tokenize_string(_sentence)[:MAX_LENGTH], MAX_LENGTH)

def encode_row(_label):
    return 1 if _label == 'positive' else 0

def encode_and_tokenize(_dataframe):

    tttt = _dataframe.apply(lambda row: tokenize_row(row['review']), axis = 1)
    llll = _dataframe.apply(lambda row: encode_row(row['sentiment']), axis = 1)

    data_dict = { 'label': llll, 'tokens': tttt }

    encoded_and_tokenized = pd.DataFrame(data_dict, columns = ['label', 'tokens'])

    return encoded_and_tokenized

In [0]:
train_df_tokenized = encode_and_tokenize(train_df)
val_df_tokenized = encode_and_tokenize(val_df)
test_df_tokenized = encode_and_tokenize(test_df)

In [0]:
from collections import Counter

def get_tokens_list(_dataframes_list):
    
    all_dataframe_ = pd.concat(_dataframes_list)
    
    all_tokens_ = []
    
    for sent_ in all_dataframe_['tokens'].values:
        all_tokens_.extend(sent_)

    tokens_counter_ = Counter(all_tokens_)
                
    return [t for t, _ in tokens_counter_.most_common(LIMIT_OF_TOKENS)]

In [0]:
tokens_list = get_tokens_list([train_df_tokenized, val_df_tokenized, test_df_tokenized])

In [0]:
word_to_int_dict = {}

word_to_int_dict.update(
    {t : i for i, t in enumerate(tokens_list)})

In [0]:
def intize_row(_tokens):
    return np.array([word_to_int_dict[t]
                if t in word_to_int_dict
                else 0
            for t in _tokens])

def encode_and_tokenize(_dataframe):

    iiii = _dataframe.apply(lambda row: intize_row(row['tokens']), axis = 1)

    data_dict = { 'label': _dataframe['label'], 'ints': iiii }

    intized = pd.DataFrame(data_dict, columns = ['label', 'ints'])

    return intized

In [0]:
train_df_intized = encode_and_tokenize(train_df_tokenized)
val_df_intized = encode_and_tokenize(val_df_tokenized)
test_df_intized = encode_and_tokenize(test_df_tokenized)

In [19]:
train_df_intized

,label,ints
40632,0,"[1, 3177, 630, 34, 1651, 7349, 14, 1, 67, 74, ..."
47816,1,"[8, 190, 12, 32, 79, 71, 23, 39, 1798, 2058, 4..."
1573,1,"[11, 6, 62, 51, 196, 987, 10, 1, 20, 3, 2, 74,..."
2090,1,"[11, 15, 6, 158, 319, 4, 2, 452, 2409, 879, 7,..."
25406,1,"[238, 69, 7, 433, 11, 195, 5, 169, 19, 1, 395,..."
...,...,...
40447,1,"[11, 6, 27, 4, 1, 115, 110, 7, 26, 113, 98, 85..."
15087,0,"[8, 13, 7284, 9, 9, 174, 25974, 634, 7, 119, 1..."
34353,0,"[0, 109, 1086, 10, 11, 0, 11, 15, 14, 427, 7, ..."
32328,0,"[8, 13, 83, 129, 1021, 145, 150, 19, 7, 130, 3..."


### Задание 2

Реализуйте и обучите двунаправленную рекуррентную сеть (_LSTM_ или _GRU_).

Какого качества классификации удалось достичь?

In [20]:
! pip install tensorflow-gpu --pre --quiet

! pip show tensorflow-gpu

Name: tensorflow-gpu
Version: 2.2.0rc3
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: opt-einsum, gast, google-pasta, tensorboard, tensorflow-estimator, wheel, absl-py, grpcio, astunparse, wrapt, h5py, scipy, termcolor, six, numpy, protobuf, keras-preprocessing
Required-by: 


In [0]:
import tensorflow as tf
from tensorflow import keras

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout

model = tf.keras.Sequential()

model.add(Bidirectional(LSTM(MAX_LENGTH, return_sequences = False, recurrent_dropout = 0.2), merge_mode = 'concat',
          input_shape = (MAX_LENGTH, 1)))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))

In [28]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 400)               323200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 401       
Total params: 323,601
Trainable params: 323,601
Non-trainable params: 0
_________________________________________________________________


In [0]:
X_train = np.asarray(list(train_df_intized['ints'].values), dtype = float)[..., np.newaxis]
X_test = np.asarray(list(test_df_intized['ints'].values), dtype = float)[..., np.newaxis]

y_train = np.asarray(list(train_df_intized['label'].values))
y_test = np.asarray(list(test_df_intized['label'].values))

In [31]:
r = 1093

r_v = 234

batch_size = 32

model.fit(x = X_train[:r * batch_size], y = y_train[:r * batch_size],
          validation_data = (X_test[:r_v * batch_size], y_test[:r_v * batch_size]),
          epochs = 5, batch_size = batch_size)

Epoch 1/5
1093/1093 [==============================] - 459s 420ms/step - loss: 0.6900 - accuracy: 0.5409 - val_loss: 0.6778 - val_accuracy: 0.5693
Epoch 2/5
1093/1093 [==============================] - 456s 417ms/step - loss: 0.6831 - accuracy: 0.5604 - val_loss: 0.6748 - val_accuracy: 0.5829
Epoch 3/5
1093/1093 [==============================] - 457s 418ms/step - loss: 0.6810 - accuracy: 0.5654 - val_loss: 0.6875 - val_accuracy: 0.5469
Epoch 4/5
1093/1093 [==============================] - 457s 418ms/step - loss: 0.6794 - accuracy: 0.5711 - val_loss: 0.6742 - val_accuracy: 0.5773
Epoch 5/5
1093/1093 [==============================] - 462s 422ms/step - loss: 0.6768 - accuracy: 0.5775 - val_loss: 0.6725 - val_accuracy: 0.5804


### Задание 3

Используйте индексы слов и их различное внутреннее представление (_word2vec_, _glove_). Как влияет данное преобразование на качество классификации?

### Задание 4

Поэкспериментируйте со структурой сети (добавьте больше рекуррентных, полносвязных или сверточных слоев). Как это повлияло на качество классификации?

### Задание 5

Используйте предобученную рекуррентную нейронную сеть (например, _DeepMoji_ или что-то подобное).

Какой максимальный результат удалось получить на контрольной выборке?